In [4]:
from google.colab import drive # type: ignore
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu114
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes triton
!pip install transformers datasets
!pip install -U bitsandbytes


Looking in indexes: https://download.pytorch.org/whl/cu114
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.9 MB/s eta 0:00:00


In [6]:
!pip install "unsloth[torch,accelerate,trl]"
!pip install unsloth-zoo # Install the missing dependency

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [7]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
import pandas as pd
from trl import SFTTrainer
from transformers import TrainingArguments
import unicodedata
import os
import re

# Desativar o logging do wandb
os.environ["WANDB_DISABLED"] = "true"

TRAIN_DATA_PATH = "/content/drive/MyDrive/tc3-final/trn.json"
TEST_DATA_PATH = "/content/drive/MyDrive/tc3-final/tst.json"

OUTPUT_SANITIZED_TRAIN_DATA_PATH = "/content/drive/MyDrive/tc3-final/sanitized_trn.json"
OUTPUT_SANITIZED_TEST_DATA_PATH = "/content/drive/MyDrive/tc3-final/sanitized_tst.json"

OUTPUT_TRAIN_DATASET = "/content/drive/MyDrive/tc3-final/formatted_train_dataset.json"
OUTPUT_TEST_DATASET = "/content/drive/MyDrive/tc3-final/formatted_test_dataset.json"


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
def sanitize_dataset(input_file_path, output_file_path):
  # Carregar o arquivo original
  with open(input_file_path, "r") as file:
      data = [json.loads(line) for line in file]

  # Filtrar as colunas "title" e "content"
  filtered_data = [{"title": item["title"], "content": item["content"]} for item in data]

  # Contagem de vazios, nulos e duplicados
  empty_titles = 0
  null_titles = 0
  empty_contents = 0
  null_contents = 0
  duplicate_contents = 0

  # Rastrear duplicados na coluna "content"
  content_counts = {}

  # Contar os valores vazios, nulos e duplicados
  for item in data:
      # Contando Titles
      if item['title'] == "":
          empty_titles += 1
      elif item['title'] is None:
          null_titles += 1

      # Contando Contents
      if item['content'] == "":
          empty_contents += 1
      elif item['content'] is None:
          null_contents += 1

      # Contar duplicados na coluna "content"
      content = item['content']
      if content in content_counts:
          content_counts[content] += 1
      else:
          content_counts[content] = 1

  # Contar valores duplicados
  duplicate_contents = sum(1 for count in content_counts.values() if count > 1)

  total_titles = len(data)
  total_contents = len(data)

  # Exibir os resultados
  print(f"Total de Titles vazios: {empty_titles}")
  print(f"Total de Contents vazios: {empty_contents}")
  print(f"Total de Titles: {total_titles}")
  print(f"Total de Contents: {total_contents}")
  print(f"Total de Contents duplicados: {duplicate_contents}")

  # Retirar as colunas "title" e "content" vazias
  filtered_data = [item for item in data if item['title'] and item['content']]

  # Remover linhas com valores duplicados na coluna "content"
  seen_contents = set()
  duplicated_data = []
  for item in filtered_data:
      if item['content'] not in seen_contents:
          duplicated_data.append(item)
          seen_contents.add(item['content'])

  # Contar duplicados após a remoção
  duplicated_content_counts = {}
  for item in duplicated_data:
      content = item['content']
      if content in duplicated_content_counts:
          duplicated_content_counts[content] += 1
      else:
          duplicated_content_counts[content] = 1

  remaining_duplicates = sum(1 for count in duplicated_content_counts.values() if count > 1)

  # Contagem final
  empty_titles = 0
  null_titles = 0
  empty_contents = 0
  null_contents = 0

  for item in duplicated_data:
      # Contando Titles
      if item['title'] == "":
          empty_titles += 1
      elif item['title'] is None:
          null_titles += 1

      # Contando Contents
      if item['content'] == "":
          empty_contents += 1
      elif item['content'] is None:
          null_contents += 1

  total_titles = len(duplicated_data)
  total_contents = len(duplicated_data)

  print(f"------------------------Após a limpeza------------------------")
  # Exibir os resultados separados
  print(f"Total de Titles vazios: {empty_titles}")
  print(f"Total de Contents vazios: {empty_contents}")
  print(f"Total de Titles: {total_titles}")
  print(f"Total de Contents: {total_contents}")
  print(f"Total de Contents duplicados restantes: {remaining_duplicates}")

  # Salvar em um novo arquivo JSON
  with open(output_file_path, "w") as outfile:
      json.dump(duplicated_data, outfile, indent=4)

  print(f"Novo arquivo salvo em: {output_file_path}")

In [ ]:
sanitize_dataset(TRAIN_DATA_PATH, OUTPUT_SANITIZED_TRAIN_DATA_PATH)
sanitize_dataset(TEST_DATA_PATH, OUTPUT_SANITIZED_TEST_DATA_PATH)

Total de Titles vazios: 126834
Total de Contents vazios: 749901
Total de Titles: 2248619
Total de Contents: 2248619
Total de Contents duplicados: 81091
------------------------Após a limpeza------------------------
Total de Titles vazios: 0
Total de Contents vazios: 0
Total de Titles: 1173617
Total de Contents: 1173617
Total de Contents duplicados restantes: 0
Novo arquivo salvo em: /content/drive/MyDrive/tc3-final/sanitized_trn.json
Total de Titles vazios: 55556
Total de Contents vazios: 323121
Total de Titles: 970237
Total de Contents: 970237
Total de Contents duplicados: 29423
------------------------Após a limpeza------------------------
Total de Titles vazios: 0
Total de Contents vazios: 0
Total de Titles: 531945
Total de Contents: 531945
Total de Contents duplicados restantes: 0
Novo arquivo salvo em: /content/drive/MyDrive/tc3-final/sanitized_tst.json


In [9]:
# Função para processar e formatar o dataset
def processar_formatar_dataset(input_file_path, output_file_path, sample_size=None):

    print(f"Carregando dataset de: {input_file_path}")

    # Carrega o arquivo JSON completo (array de objetos)
    with open(input_file_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)

    print(f"Total de registros carregados: {len(json_data)}")

    # Filtra apenas os registros que têm 'title' e 'content'
    data = []
    for item in json_data:
        if isinstance(item, dict) and 'title' in item and 'content' in item:
            title = item['title']
            content = item['content']
            data.append({'title': title, 'content': content})
        else:
            print(f"Registro inválido ignorado: {item}")

    print(f"Registros válidos encontrados: {len(data)}")

    # Converte a lista de dicionários em um DataFrame do pandas
    df = pd.DataFrame(data)

    # Função para normalizar o texto
    def normalize_text(text):
        # Check if text is a string before processing
        if not isinstance(text, str):
            return ""
        # Remove caracteres de controle
        text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
        # Remove espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        return text


    # Aplica a função de normalização nas colunas 'title' e 'content'
    df['title'] = df['title'].apply(normalize_text)
    df['content'] = df['content'].apply(normalize_text)

    # Cria as colunas 'instruction', 'input' e 'output' para o formato necessário
    df['instruction'] = "Answer the question based on the product title."
    df['input'] = df['title'].apply(lambda x: f"What is '{x}'?")
    df['output'] = df['content']

    # Se 'sample_size' estiver definido, pega uma amostra do DataFrame
    if sample_size is not None:
        df = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    # Converte para uma lista de dicionários
    formatted_data = df[['instruction', 'input', 'output']].to_dict(orient='records')

    # Salva o resultado em um arquivo JSON
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, ensure_ascii=False, indent=4)

    print(f"Dataset processado e salvo em '{output_file_path}'. Total de registros: {len(df)}")

In [10]:
# Mantendo o tamanho original do dataset conforme solicitado
processar_formatar_dataset(OUTPUT_SANITIZED_TRAIN_DATA_PATH, OUTPUT_TRAIN_DATASET, sample_size=150000)
processar_formatar_dataset(OUTPUT_SANITIZED_TEST_DATA_PATH, OUTPUT_TEST_DATASET, sample_size=30000)

Carregando dataset de: /content/drive/MyDrive/tc3-final/sanitized_trn.json
Total de registros carregados: 1173617
Registros válidos encontrados: 1173617
Dataset processado e salvo em '/content/drive/MyDrive/tc3-final/formatted_train_dataset.json'. Total de registros: 150000
Carregando dataset de: /content/drive/MyDrive/tc3-final/sanitized_tst.json
Total de registros carregados: 531945
Registros válidos encontrados: 531945
Dataset processado e salvo em '/content/drive/MyDrive/tc3-final/formatted_test_dataset.json'. Total de registros: 30000


In [11]:
train_dataset = load_dataset("json", data_files=OUTPUT_TRAIN_DATASET, split="train")
test_dataset = load_dataset("json", data_files=OUTPUT_TEST_DATASET, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
# Define o template de prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [13]:
# Função para formatar os prompts
def formatar_prompts(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Formata o texto usando o template
        text = alpaca_prompt.format(instruction, input_text, output)
        texts.append(text)
    return { "text": texts }

In [14]:
# Aplica a formatação nos datasets de treinamento e teste
train_dataset = train_dataset.map(formatar_prompts, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(formatar_prompts, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [15]:
# Configurações otimizadas do modelo
max_seq_length = 128  # Reduzido de 256 para 128 (mais rápido)
dtype = torch.bfloat16
load_in_4bit = True

In [16]:
# Carrega o modelo e o tokenizer usando o unsloth

model_name = "unsloth/llama-3.2-3b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device does not support bfloat16. Will change to float16.


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [17]:
# Configura o modelo para Fine-Tuning com LoRA otimizado
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Reduzido de 16 para 8 (mais rápido)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=8,  # Reduzido de 16 para 8
    lora_dropout=0.1,  # Adicionado dropout para regularização
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [37]:
def config_training():
    return TrainingArguments(
        output_dir="./results_llama_4bit_optimized",
        num_train_epochs=1,
        per_device_train_batch_size=32,  # ULTRA: 32 (deve usar ~12-13GB RAM)
        gradient_accumulation_steps=1,   # Batch efetivo = 32
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        load_best_model_at_end=True,
        eval_strategy="steps",
        eval_steps=1000,  # Avalia menos frequentemente para velocidade
        save_strategy="steps",
        save_steps=1000,  # Salva menos frequentemente
        save_total_limit=2,
        logging_steps=200,  # Logs mais frequentes para monitorar
        learning_rate=3e-4,  # Learning rate ajustado para batch maior
        fp16=True,
        optim="adamw_8bit",
        report_to="none",
        dataloader_num_workers=0,
        remove_unused_columns=False,
        max_grad_norm=1.0,
        warmup_ratio=0.01,  # Warmup mínimo
        lr_scheduler_type="linear",
        dataloader_pin_memory=False,
        dataloader_drop_last=True,
        gradient_checkpointing=False,  # Desabilitado para velocidade
        ddp_find_unused_parameters=False,
    )

In [24]:
# Configurações adicionais para otimização de memória e velocidade
import torch

# Otimizações de memória
torch.backends.cudnn.benchmark = True  # Otimização para CNNs (pode ajudar)
torch.backends.cuda.matmul.allow_tf32 = True  # Permitir TF32 para velocidade
torch.backends.cudnn.allow_tf32 = True

# Configurações de garbage collection
import gc
gc.collect()
torch.cuda.empty_cache()

print("Configurações de otimização aplicadas!")


Configurações de otimização aplicadas!


In [38]:
# Configurações adicionais do tokenizador para resolver o erro
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

training_args = config_training()

# Define o Trainer usando o SFTTrainer com configurações corrigidas
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, # Added the evaluation dataset
    args=training_args,
    max_seq_length=max_seq_length,
    packing=True, # Changed to True to handle varying sequence lengths
    dataset_text_field="text",
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [27]:
# Função para recuperar treinamento interrompido (útil para Colab)
def resume_training_from_checkpoint(checkpoint_path=None):
    """
    Função para retomar o treinamento de onde parou
    Use esta função se o Colab desconectar durante o treino
    """
    if checkpoint_path is None:
        # Lista checkpoints disponíveis
        import os
        checkpoint_dir = "./results_llama_4bit_optimized"
        if os.path.exists(checkpoint_dir):
            checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")]
            if checkpoints:
                # Pega o checkpoint mais recente
                latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))[-1]
                checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
                print(f"Checkpoint mais recente encontrado: {checkpoint_path}")
            else:
                print("Nenhum checkpoint encontrado. Iniciando treinamento do zero.")
                return None
        else:
            print("Diretório de resultados não encontrado. Iniciando treinamento do zero.")
            return None

    return checkpoint_path


In [28]:
# Treinamento com recuperação automática de checkpoints
try:
    # Tenta recuperar checkpoint mais recente
    checkpoint = resume_training_from_checkpoint()
    if checkpoint:
        print(f"Retomando treinamento do checkpoint: {checkpoint}")
        trainer.train(resume_from_checkpoint=checkpoint)
    else:
        print("Iniciando treinamento do zero...")
        trainer.train()
except Exception as e:
    print(f"Erro durante o treinamento: {e}")
    print("Tentando continuar sem checkpoint...")
    trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 244,096 | Num Epochs = 1 | Total steps = 7,628
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 12,156,928 of 3,224,906,752 (0.38% trained)


Nenhum checkpoint encontrado. Iniciando treinamento do zero.
Iniciando treinamento do zero...


Step,Training Loss,Validation Loss
1000,2.300800,2.293093
2000,2.299400,2.290492
3000,2.282800,2.290546
4000,2.276000,2.287428
5000,2.276100,2.284550
6000,2.282600,2.283473
7000,2.281800,2.284083


In [29]:
# Prepara o modelo para inferência
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
          

In [30]:
# Salva o modelo treinado
model.save_pretrained("/content/drive/MyDrive/tc3-final/trained_model_llama_4bit_demo")
tokenizer.save_pretrained("/content/drive/MyDrive/tc3-final/trained_model_llama_4bit_demo")

print("Treinamento concluído e modelo salvo.")

Treinamento concluído e modelo salvo.


In [33]:
# Teste do Modelo Treinado
# Carrega o modelo treinado
from unsloth import FastLanguageModel
import torch
import time

model_name = "/content/drive/MyDrive/tc3-final/trained_model_llama_4bit_demo"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)

# Prepara um prompt de teste
prompt = alpaca_prompt.format(
    "Answer the question based on the product title.",
    "What is 'Girls Ballet Tutu Neon Pink'?",
    ""
)

# Função para testar o modelo com diferentes exemplos
def test_model(instruction, input_text, max_new_tokens=100, temperature=0.7):
    prompt = alpaca_prompt.format(instruction, input_text, "")

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        start_time = time.time()
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        generation_time = time.time() - start_time

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extrai apenas a resposta (remove o prompt)
    response = generated_text.split("### Response:")[-1].strip()

    return response, generation_time

# Lista de exemplos de teste diversificados
test_examples = [
    {
        "category": "Roupas Femininas",
        "title": "Women's Summer Dress Floral Print",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Women's Summer Dress Floral Print'?"
    },
    {
        "category": "Eletrônicos",
        "title": "Wireless Bluetooth Headphones Noise Cancelling",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Wireless Bluetooth Headphones Noise Cancelling'?"
    },
    {
        "category": "Casa e Jardim",
        "title": "Stainless Steel Kitchen Knife Set 6 Pieces",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Stainless Steel Kitchen Knife Set 6 Pieces'?"
    },
    {
        "category": "Esportes",
        "title": "Men's Running Shoes Lightweight Athletic",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Men's Running Shoes Lightweight Athletic'?"
    },
    {
        "category": "Livros",
        "title": "Python Programming for Beginners Complete Guide",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Python Programming for Beginners Complete Guide'?"
    },
    {
        "category": "Eletrônicos",
        "title": "Smart TV 50 inch 4K UHD",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Smart TV 50 inch 4K UHD'?"
    },
    {
        "category": "Beleza e Saúde",
        "title": "Organic Argan Oil 100% Pure",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Organic Argan Oil 100% Pure'?"
    },
    {
        "category": "Brinquedos e Jogos",
        "title": "Educational Building Blocks for Kids Age 3+",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Educational Building Blocks for Kids Age 3+'?"
    },
    {
        "category": "Automotivo",
        "title": "Car Wax Kit with Microfiber Towels",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Car Wax Kit with Microfiber Towels'?"
    },
    {
        "category": "Joias",
        "title": "Sterling Silver Pendant Necklace with Blue Gemstone",
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Sterling Silver Pendant Necklace with Blue Gemstone'?"
    }
]

print("=" * 80)
print("TESTE ABRANGENTE DO MODELO TREINADO")
print("=" * 80)

# Testa cada exemplo
for i, example in enumerate(test_examples, 1):
    print(f"\nTESTE {i}: {example['category']}")
    print(f"Título: {example['title']}")
    print(f"Pergunta: {example['input']}")
    print("-" * 60)

    try:
        response, gen_time = test_model(
            example['instruction'],
            example['input'],
            max_new_tokens=150,
            temperature=0.7
        )

        print(f"Resposta: {response}")

    except Exception as e:
        print(f"Erro no teste: {e}")

    print("-" * 60)

print("\n" + "=" * 80)
print("TESTE CONCLUÍDO")
print("=" * 80)

==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/llama-3.2-3b-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
TESTE ABRANGENTE DO MODELO TREINADO

TESTE 1: Roupas Femininas
Título: Women's Summer Dress Floral Print
Pergunta: What is 'Women's Summer Dress Floral Print'?
------------------------------------------------------------
Resposta: Women's Summer Dress Floral Print
------------------------------------------------------------

TESTE 2: Eletrônicos
Título: Wireless Bluetooth Headphones Noise Cancelling
Pergunta: What is 'Wireless Blu